In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
raw_sentence = ["i like dog", "i love coffee", "i hate milk"]

In [3]:
def make_batch(sentences):
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()
        input = [word2id[n] for n in word[:-1]]
        target = word2id[word[-1]]

        input_batch.append(input)
        target_batch.append(target)

    return input_batch, target_batch

In [4]:
word_list = " ".join(raw_sentence).split()
word_list = list(set(word_list))
word2id = {w: i for i, w in enumerate(word_list)}
id2word = {i: w for i, w in enumerate(word_list)}
n_class = len(word2id)  # number of Vocabulary

n_class

7

In [5]:
class NNLM(nn.Module):
    def __init__(self):
        super(NNLM, self).__init__()
        self.embeddings = nn.Embedding(n_class, m) #embedding layer or look up table

        self.hidden1 = nn.Linear(n_step * m, n_hidden, bias=False)
        self.ones = nn.Parameter(torch.ones(n_hidden))

        self.hidden2 = nn.Linear(n_hidden, n_class, bias=False)
        self.hidden3 = nn.Linear(n_step * m, n_class, bias=False) #final layer

        self.bias = nn.Parameter(torch.ones(n_class))

    def forward(self, X):
        word_embeds = self.embeddings(X) # embeddings
        X = word_embeds.view(-1, n_step * m) # first layer
        tanh = torch.tanh(self.ones + self.hidden1(X)) # tanh layer
        output = self.bias + self.hidden3(X) + self.hidden2(tanh) # summing up all the layers with bias
        return word_embeds, output



In [6]:
n_step = 2
n_hidden = 2
m = 2

In [7]:
model = NNLM()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
input_batch, target_batch = make_batch(raw_sentence)
input_batch = torch.LongTensor(input_batch)
target_batch = torch.LongTensor(target_batch)

In [10]:
for epoch in range(5000):
    optimizer.zero_grad()
    embeddings, output = model(input_batch)

    # output : [batch_size, n_class], target_batch : [batch_size]
    loss = criterion(output, target_batch)
    if (epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()



Epoch: 1000 cost = 0.000532
Epoch: 2000 cost = 0.000293
Epoch: 3000 cost = 0.000164
Epoch: 4000 cost = 0.000093
Epoch: 5000 cost = 0.000053


In [11]:
# Predict
predict = model(input_batch).data.max(1, keepdim=True)[1]



AttributeError: 'tuple' object has no attribute 'data'

In [12]:
# Test
print([sen.split()[:2] for sen in raw_sentence], '->', [id2word[n.item()] for n in predict.squeeze()])

NameError: name 'predict' is not defined

In [13]:
embeddings

tensor([[[-2.7784,  0.2981],
         [-2.5415,  1.8039]],

        [[-2.7784,  0.2981],
         [ 1.8503,  1.1788]],

        [[-2.7784,  0.2981],
         [-0.7446, -2.3872]]], grad_fn=<EmbeddingBackward0>)